In [1]:
import ee
import geemap

In [3]:
ee.Authenticate()#重新连接端口

Enter verification code:  4/1AbUR2VNJTOlQyIrvfN93ZlOgneTleJFMANr0g3QUfK1hTEhBUXdXuXO6UzI



Successfully saved authorization token.


In [2]:
ee.Initialize()#重置刷新

In [3]:
dxal = geemap.Map(center=[51.87,124.36],zoom=7,height=600)
dxal.add_basemap('ROADMAP')
#载入矢量数据集
data = r'C:\Users\R6meg\Desktop\UTF8\bianjie.shp'
sl = geemap.shp_to_ee(data)
dxal.addLayer(sl,{},'DXAL')
# 依据字段筛选矢量数据
# data0 = r'C:\Users\R6meg\Desktop\tree\point_all.shp'
data1 = r'C:\Users\R6meg\Desktop\tree\GEE_train.shp'
data2 = r'C:\Users\R6meg\Desktop\tree\GEE_test.shp'
data3 = r'C:\Users\R6meg\Desktop\tree\train_add_maxent.shp'
data4 = r'C:\Users\R6meg\Desktop\tree\test_add_maxent.shp'
# yd0 = geemap.shp_to_ee(data0)
yd1 = geemap.shp_to_ee(data1)
yd2 = geemap.shp_to_ee(data2)
yd3 = geemap.shp_to_ee(data3)
yd4 = geemap.shp_to_ee(data4)
# dxal.addLayer(yd0,{},'yangdi_all')
dxal.addLayer(yd1,{},'GEE_train')
dxal.addLayer(yd2,{},'GEE_test')
dxal.addLayer(yd1,{},'train_add_maxent')
dxal.addLayer(yd2,{},'test_add_maxent')
# 载入Sentinel2影像
S2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
# 根据矢量边界与日期筛选影像
S2_2022 = S2.filterBounds(sl).filterDate('2022-05-01','2022-09-30')
#载入哨兵2云处理影像
S2_Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
S2_Clouds_2022 = S2_Clouds.filterBounds(sl).filterDate('2022-05-01','2022-09-30')
###############################################编写去云函数
def maskEdges(s2_img):
    return s2_img.updateMask(s2_img.select('B8A').mask()
    .updateMask(s2_img.select('B9').mask()))
def maskClouds(img):
    clouds = ee.Image(img.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(5)#最大云概率为5
    return img.updateMask(isNotCloud)
S2_2022_image = S2_2022.map(maskEdges)
s2SrWithCloudMask_2022 = ee.Join.saveFirst('cloud_mask').apply(**{
    "primary": S2_2022_image,
    "secondary": S2_Clouds_2022,
    "condition": ee.Filter.equals(**{
     "leftField": "system:index", 
     "rightField":"system:index"})
     })
s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask_2022).map(maskClouds)
######################################################################## 定义函数计算 NDVI
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi).set('system:time_start', image.get('system:time_start'))
dxal_NDVI = s2CloudMasked.map(add_ndvi)
########################################################################## 计算月度融合NDVI
monthly_ndvi = dxal_NDVI \
                        .map(lambda img: img.select(['NDVI']).set('system:time_start', img.get('system:time_start'))) \
                            .map(lambda img: img.set('month', img.date().get('month')))                   
temp = ee.ImageCollection(monthly_ndvi.filter(ee.Filter.calendarRange(5, 5, 'month')).mean() \
    .set("system:time_start", ee.Date.fromYMD(2022, 5, 1).millis()) \
    .set("system:index", '2022_5_1')
)
# 循环遍历每个月份
for j in range(6, 10):
    # 选取当前月份的影像并计算平均值
    data_collection = monthly_ndvi.filter(ee.Filter.calendarRange(j, j, 'month')).mean() \
                      .set('system:time_start', ee.Date.fromYMD(2022, j, 1).millis())\
                       .set("system:index", '2022_'+str(j)+'_'+str(1))
    # 合并当前月份的影像到月度合成图像中
    temp = temp.merge(data_collection)
#####################################################################################################################插值
def func_won(image):
  currentDate = ee.Date(image.get('system:time_start'))
  meanImage = temp.filterDate(currentDate.advance(-2,'month'), currentDate.advance(2, 'month')).mean()
  return meanImage.where(image, image)
replacedVals = temp.map(func_won)
replacedVals_a = replacedVals.filterBounds(sl).toBands()
def stackCollection(collection):
    first = ee.Image(collection.first()).select([])
    def appendBands(image, previous):
        return ee.Image(previous).addBands(image)
    return ee.Image(collection.iterate(appendBands, first))
stacked = stackCollection(replacedVals)
stacked = stacked.select('NDVI', 'NDVI_1', 'NDVI_2', 'NDVI_3', 'NDVI_4') \
                 .rename('NDVI_May','NDVI_June','NDVI_July','NDVI_Aug','NDVI_Sep')
################################################TVI
def addTVI(image):
    tvi = image.expression(
        '(NIR - G) / (NIR + 0.5 * G - 1.5 * B)',
        {   'NIR': image.select('B8').divide(10000), # 近红外波段
            'G': image.select('B3').divide(10000), # 绿色波段
            'B': image.select('B2').divide(10000), # 蓝色波段
}).rename('TVI')
    return image.addBands(tvi)
dxal_tvi = dxal_NDVI.map(addTVI)
########################################################MTCI
def addMTCI(image):
    mtci = image.expression('(RE2 - RE1)/ (RE1 - Red)', {
    'RE2': image.select('B6').divide(10000),
    'RE1': image.select('B5').divide(10000),
    'Red': image.select('B4').divide(10000)
    })
    return image.addBands(mtci.rename('MTCI'))
dxal_MTCI = dxal_tvi.map(addMTCI)
########################################################NBR
def add_nbr(image):
    nbr = image.normalizedDifference(['B8', 'B12']).rename('NBR')
    return image.addBands(nbr)
dxal_NBR = dxal_MTCI.map(add_nbr)
############################################################IRECI
def addIRECI(image):
    ireci = image.expression('(RE3 - R1)/ (RE1 - RE2)', {
    'RE3': image.select('B7'),
    'R1': image.select('B4'),
    'RE1': image.select('B5'),
    'RE2': image.select('B6')
    }).float()
    return image.addBands(ireci.rename('IRECI'))
dxal_IRECI = dxal_NBR.map(addIRECI)
#############################################################3MCARI2
def addMCARI2(image):
    mcari2 = image.expression('((RE2 - RE1) - 0.2 * (RE2-Green)) * (RE2/ RE1)', {
    'RE2': image.select('B6').divide(10000),
    'RE1': image.select('B5').divide(10000),
    'Green': image.select('B3').divide(10000)
    })
    REP = image.expression('705+35*(0.5*(RED + RE3) - RE1) / (RE2 - RE1)', {
    'RED': image.select('B4'),
    'RE1': image.select('B5'),
    'RE2': image.select('B6'),
    'RE3': image.select('B7')
    })
    CIrededge = image.expression(('RE3 / RE1 -1'),{
        'RE3':image.select('B7'),
        'RE1':image.select('B5')
    })
    return image.addBands(mcari2.rename('MCARI2')) \
                .addBands(REP.rename('REP')) \
                .addBands(CIrededge.rename('CIrededge'))
dxal_MCARI2 = dxal_IRECI.map(addMCARI2)
###############################LSWI
def add_LSWI(image):
    lswi = image.normalizedDifference(['B8', 'B11']).rename('LSWI')
    return image.addBands(lswi)
dxal_LSWI = dxal_MCARI2.map(add_LSWI)
###############################NDRE
def add_NDRE(image):
    NDRE = image.normalizedDifference(['B8', 'B5']).rename('NDRE')
    Chl_NDI = image.normalizedDifference(['B6', 'B5']).rename('Chl_NDI')
    NDSVI = image.normalizedDifference(['B11', 'B4']).rename('NDSVI')
    NDTI = image.normalizedDifference(['B11', 'B6']).rename('NDTI')
    NDSI = image.normalizedDifference(['B11', 'B12']).rename('NDSI')
    NDBI = image.normalizedDifference(['B11' ,'B8']).rename('NDBI')
    NDWI = image.normalizedDifference(['B3' ,'B8']).rename('NDWI')
    return image.addBands(NDRE) \
                .addBands(Chl_NDI) \
                .addBands(NDSVI) \
                .addBands(NDTI) \
                .addBands(NDSI) \
                .addBands(NDBI) \
                .addBands(NDWI)
dxal_NDRE = dxal_LSWI.map(add_NDRE)
###########################################################################定义月度NDVI波段融合到原始影像
dxal_NDVIS = dxal_NDRE.median().clipToCollection(sl)
###########################计算GLCM灰度共生矩阵
bands = dxal_NDVIS.select('B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12')
glcm = bands.divide(10000).unitScale(0,1).multiply(255).toInt16().glcmTexture(size = 3)
glcm_bands = glcm.select(
                         'B2_diss','B2_var','B2_savg','B2_shade','B2_corr','B3_diss','B3_var','B3_savg','B3_shade','B3_corr',
                         'B4_diss','B4_var','B4_savg','B4_shade','B4_corr','B5_diss','B5_var','B5_savg','B5_shade','B5_corr',
                         'B6_diss','B6_var','B6_savg','B6_shade','B6_corr','B7_diss','B7_var','B7_savg','B7_shade','B7_corr',
                         'B8_diss','B8_var','B8_savg','B8_shade','B8_corr','B8A_diss','B8A_var','B8A_savg','B8A_shade','B8A_corr',
                         'B11_diss','B11_var','B11_savg','B11_shade','B11_corr','B12_diss','B12_var','B12_savg','B12_shade','B12_corr'
)
con_bands = glcm.select('B2_contrast','B3_contrast','B4_contrast','B5_contrast','B6_contrast','B7_contrast','B8_contrast','B8A_contrast','B11_contrast','B12_contrast') \
                .rename('B2_con','B3_con','B4_con','B5_con','B6_con','B7_con','B8_con','B8A_con','B11_con','B12_con')
###############################################计算缨帽变换
coefficients = ee.Array([  
  [0.0356, 0.0822, 0.1360, 0.2611, 0.2964, 0.3338, 0.3877, 0.3895, 0.0949, 0.3882, 0.1366, 0.4750],  
  [-0.0635, -0.1128, -0.1680, -0.3480, -0.3303, 0.0852, 0.3302, 0.3165, 0.0467, -0.4578,  -0.4064, 0.3625],  
  [0.0649, 0.1363, 0.2802, 0.3072, 0.5288, 0.1379, -0.0001, -0.0807, -0.0302,-0.4064, -0.5602,-0.1389]
])
arrayimageID1 = dxal_NDVIS.select(['B1','B2','B3','B4','B5','B6','B7','B8','B9','B11','B12','B8A']).toArray()
arrayImageID2 = arrayimageID1.toArray(1)
componentsImage = ee.Image(coefficients) \
                    .matrixMultiply(arrayImageID2) \
                    .arrayProject([0]) \
                    .arrayFlatten([['brightness', 'greenness', 'wetness']])
vis_ktchange = {
    'min' : -0.1,
    'max' : [0.5, 0.1, 0.1],
    'bands' : ['brightness', 'greenness', 'wetness']
}
dxal.addLayer(componentsImage,vis_ktchange,'componentsImage')
##############################################加入maxent分类数据
birch_image = ee.Image('projects/gaogao-gee-learn/assets/0_avg')
larch_image = ee.Image('projects/gaogao-gee-learn/assets/1_avg')
Mixed_coniferous_forest_image = ee.Image('projects/gaogao-gee-learn/assets/2_avg')
mixed_broadleaf_forest_image = ee.Image('projects/gaogao-gee-learn/assets/3_avg')
mixed_coniferous_and_broadleaf_forest_image = ee.Image('projects/gaogao-gee-learn/assets/4_avg')
birch_band = birch_image.select('b1').rename('birch')
larch_band = larch_image.select('b1').rename('larch')
Mixed_coniferous_forest_band = Mixed_coniferous_forest_image.select('b1').rename('Mixed_2')
mixed_broadleaf_forest_band = mixed_broadleaf_forest_image.select('b1').rename('mixed_3')
mixed_coniferous_and_broadleaf_forest_band = mixed_coniferous_and_broadleaf_forest_image.select('b1').rename('mixed_4')
#############################################计算海拔坡度坡向
DEM = ee.Image("USGS/SRTMGL1_003").clip(sl)
elevation = ee.Terrain.products(DEM).select('elevation')
slope = ee.Terrain.slope(DEM)
aspect = ee.Terrain.aspect(DEM)
###################################
dxal_s2_image = dxal_NDVIS.addBands(stacked) \
                          .addBands(glcm_bands) \
                          .addBands(componentsImage) \
                          .addBands(elevation) \
                          .addBands(slope) \
                          .addBands(aspect) \
                          .addBands(con_bands) \
                          .addBands(birch_band) \
                          .addBands(larch_band) \
                          .addBands(Mixed_coniferous_forest_band) \
                          .addBands(mixed_broadleaf_forest_band) \
                          .addBands(mixed_coniferous_and_broadleaf_forest_band)                                                
rgbVis = {"min": 0, "max": 3000, "bands": ['B4', 'B3', 'B2']};
dxal.addLayer(dxal_s2_image,rgbVis,"S2_NOCLOUD")
classified_image = ee.Image('projects/gaogao-gee-learn/assets/maxent_fin')
dxal.addLayer(classified_image,{},"classified_image")
dxal

Map(center=[51.87, 124.36], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

In [4]:
##############################################最小噪声变换
import numpy as np
from sklearn.decomposition import PCA

# 定义函数将图像转换为像素级数据
def convertToPixels(image):
    # 选择波段并转换为数组
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A','B11', 'B12',
             'NDVI', 'TVI', 'MTCI', 'NBR', 'IRECI', 'MCARI2', 'REP', 'CIrededge', 'LSWI', 'NDRE', 'Chl_NDI', 'NDSVI', 'NDTI', 'NDSI', 'NDBI', 'NDWI'
             ]  # 替换为你要使用的特征波段的名称
    array = image.select(bands).toFloat().toArray()  
    # 添加像素坐标作为属性
    coords = ee.Image.pixelCoordinates(array.projection()).addBands(array)
    # 转换为像素级数据
    return coords.reduceRegion(reducer=ee.Reducer.toList(), geometry=image.geometry(), scale=30)

image_collection = dxal_NDRE

# 初始化一个空的特征矩阵列表
feature_matrix_list = []

# 遍历图像集合，将每个图像转换为像素级数据并添加到特征矩阵列表
for image in dxal_NDRE.toList(dxal_NDRE.size()).getInfo():
    pixel_values = np.array(convertToPixels(ee.Image(image)).getInfo().values())
    feature_matrix_list.append(pixel_values)

# 将特征矩阵列表转换为NumPy数组
feature_matrix = np.concatenate(feature_matrix_list, axis=0)

# 执行最小噪声变换（PCA）
pca = PCA(n_components=3)  # 替换为你想要的主成分数量
principal_components = pca.fit_transform(feature_matrix)

EEException: Unrecognized argument type to convert to an Image: {'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [1830, 1830], 'crs': 'EPSG:32651', 'crs_transform': [60, 0, 399960, 0, -60, 5700000]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32651', 'crs_transform': [10, 0, 399960, 0, -10, 5700000]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32651', 'crs_transform': [10, 0, 399960, 0, -10, 5700000]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32651', 'crs_transform': [10, 0, 399960, 0, -10, 5700000]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [5490, 5490], 'crs'

In [4]:
print(dxal_s2_image.bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'NDVI', 'TVI', 'MTCI', 'NBR', 'IRECI', 'MCARI2', 'REP', 'CIrededge', 'LSWI', 'NDRE', 'Chl_NDI', 'NDSVI', 'NDTI', 'NDSI', 'NDBI', 'NDWI', 'NDVI_May', 'NDVI_June', 'NDVI_July', 'NDVI_Aug', 'NDVI_Sep', 'B2_diss', 'B2_var', 'B2_savg', 'B2_shade', 'B2_corr', 'B3_diss', 'B3_var', 'B3_savg', 'B3_shade', 'B3_corr', 'B4_diss', 'B4_var', 'B4_savg', 'B4_shade', 'B4_corr', 'B5_diss', 'B5_var', 'B5_savg', 'B5_shade', 'B5_corr', 'B6_diss', 'B6_var', 'B6_savg', 'B6_shade', 'B6_corr', 'B7_diss', 'B7_var', 'B7_savg', 'B7_shade', 'B7_corr', 'B8_diss', 'B8_var', 'B8_savg', 'B8_shade', 'B8_corr', 'B8A_diss', 'B8A_var', 'B8A_savg', 'B8A_shade', 'B8A_corr', 'B11_diss', 'B11_var', 'B11_savg', 'B11_shade', 'B11_corr', 'B12_diss', 'B12_var', 'B12_savg', 'B12_shade', 'B12_corr', 'brightness', 'greenness', 'wetness', 'elevation', 'slope', 

In [4]:
# ################################################################################值提取到点
points = ee.FeatureCollection(yd2)
# maxent = dxal_s2_image.select('aspen', 'birch', 'larch', 'Soft_and_broad_mixed_forest', 'Quercus_forest', 'pinus_sylvestris', 'Coniferous_and_broad_leaved_mixed_forest_avg1', 'Hard_and_broad_mixed_forest', 'Mixed_coniferous_forest')
bands = dxal_s2_image.select('elevation', 'slope', 'aspect',
                             'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A','B11', 'B12',
                             'NDVI', 'TVI', 'MTCI', 'NBR', 'IRECI', 'MCARI2', 'REP', 'CIrededge', 'LSWI', 'NDRE', 'Chl_NDI', 'NDSVI', 'NDTI', 'NDSI', 'NDBI', 'NDWI',
                             'NDVI_May', 'NDVI_June', 'NDVI_July', 'NDVI_Aug', 'NDVI_Sep',
                             'B2_con', 'B2_diss', 'B2_var', 'B2_savg', 'B2_shade', 'B2_corr', 
                             'B3_con', 'B3_diss', 'B3_var', 'B3_savg', 'B3_shade', 'B3_corr', 
                             'B4_con', 'B4_diss', 'B4_var', 'B4_savg', 'B4_shade', 'B4_corr', 
                             'B5_con', 'B5_diss', 'B5_var', 'B5_savg', 'B5_shade', 'B5_corr', 
                             'B6_con', 'B6_diss', 'B6_var', 'B6_savg', 'B6_shade', 'B6_corr', 
                             'B7_con', 'B7_diss', 'B7_var', 'B7_savg', 'B7_shade', 'B7_corr', 
                             'B8_con', 'B8_diss', 'B8_var', 'B8_savg', 'B8_shade', 'B8_corr', 
                             'B8A_con', 'B8A_diss', 'B8A_var', 'B8A_savg', 'B8A_shade', 'B8A_corr', 
                             'B11_con', 'B11_diss', 'B11_var', 'B11_savg', 'B11_shade', 'B11_corr', 
                             'B12_con', 'B12_diss', 'B12_var', 'B12_savg', 'B12_shade', 'B12_corr', 
                             'brightness', 'greenness', 'wetness'
                             # 'birch', 'larch', 'Mixed_2', 'mixed_3', 'mixed_4'
)
geemap.extract_values_to_points(points, bands,'test.csv',30,tile_scale=2)
# # geemap.extract_values_to_points(points, maxent,'maxent.csv',30)

Generating URL ...
Please wait ...
Data downloaded to E:\jupyter\大兴安岭数据处理\test.csv


In [6]:
print(yd1.size().getInfo())
print(yd2.size().getInfo())
print(yd3.size().getInfo())
print(yd4.size().getInfo())

646
274
646
274


In [5]:
feature_names = ['elevation', 'slope', 'aspect',
                             'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A','B11', 'B12',
                             'NDVI', 'TVI', 'MTCI', 'NBR', 'IRECI', 'MCARI2', 'REP', 'CIrededge', 'LSWI', 'NDRE', 'Chl_NDI', 'NDSVI', 'NDTI', 'NDSI', 'NDBI', 'NDWI',
                             'NDVI_May', 'NDVI_June', 'NDVI_July', 'NDVI_Aug', 'NDVI_Sep',
                             'B2_con', 'B2_diss', 'B2_var', 'B2_savg', 'B2_shade', 'B2_corr', 
                             'B3_con', 'B3_diss', 'B3_var', 'B3_savg', 'B3_shade', 'B3_corr', 
                             'B4_con', 'B4_diss', 'B4_var', 'B4_savg', 'B4_shade', 'B4_corr', 
                             'B5_con', 'B5_diss', 'B5_var', 'B5_savg', 'B5_shade', 'B5_corr', 
                             'B6_con', 'B6_diss', 'B6_var', 'B6_savg', 'B6_shade', 'B6_corr', 
                             'B7_con', 'B7_diss', 'B7_var', 'B7_savg', 'B7_shade', 'B7_corr', 
                             'B8_con', 'B8_diss', 'B8_var', 'B8_savg', 'B8_shade', 'B8_corr', 
                             'B8A_con', 'B8A_diss', 'B8A_var', 'B8A_savg', 'B8A_shade', 'B8A_corr', 
                             'B11_con', 'B11_diss', 'B11_var', 'B11_savg', 'B11_shade', 'B11_corr', 
                             'B12_con', 'B12_diss', 'B12_var', 'B12_savg', 'B12_shade', 'B12_corr', 
                             'brightness', 'greenness', 'wetness']
# feature_names_30 = ['NDVI_July','B11_savg','elevation','NDVI_Sep','B8_savg',
#                     'wetness','NDVI_May','B2_shade','B2_var','aspect',
#                     'NDWI','NDVI_June','B3_shade','B5_savg','B3',
#                     'B6_corr','slope','B7_corr','B8A_corr','NDSI'] 
                    # 'TVI','B2_corr','B8A_shade','B11','B12_var',
                    # 'B4_shade','B12','B3_savg','NDRE','MCARI2',
                    # 'B3_con','REP','B6_savg','B8_shade','Chl_NDI',
                    # 'B11_shade','B6','B7_shade','B5_corr','B4']
label = 'varieties'

In [6]:
params = {
    'features': yd1,
    'classProperty': label,
    'inputProperties': feature_names
}

In [9]:
############################################################################随机森林

In [7]:
classifier_RF= ee.Classifier.smileRandomForest(150).train(**params)

In [8]:
image = dxal_s2_image.toFloat()
classified_RF = image.select(feature_names).classify(classifier_RF)
vis_class = {
    'min':'0',
    'max':'4',
    'palette': ['006400' ,'ffbb22', 'ffff4c', 'f096ff', 'fa0000']
}
dxal.addLayer(classified_RF,vis_class,'RF')
legend_keys = ['桦树林','落叶松林','针叶混交林','阔叶混交林','针阔混交林']
colors = ['006400' ,'ffbb22', 'ffff4c', 'f096ff', 'fa0000'] 
dxal.add_legend(
    title="varieties",labels=legend_keys,colors=colors, height='230px',position='bottomright'
)
dxal

Map(bottom=11143.0, center=[51.87, 124.36], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…

In [9]:
validationSample_RF = yd2.classify(classifier_RF)
validationAccuracy_RF = validationSample_RF.errorMatrix(label, 'classification')
validationAccuracy_RF.getInfo()

[[33, 10, 1, 14, 10],
 [12, 42, 0, 7, 16],
 [2, 10, 2, 0, 4],
 [8, 5, 0, 29, 8],
 [16, 24, 1, 7, 13]]

In [10]:
validationAccuracy_RF.kappa().getInfo()

0.25762131170465674

In [11]:
validationAccuracy_RF.accuracy().getInfo()

0.4343065693430657

In [12]:
validationAccuracy_RF.producersAccuracy().getInfo()

[[0.4852941176470588],
 [0.5454545454545454],
 [0.1111111111111111],
 [0.58],
 [0.21311475409836064]]

In [13]:
validationAccuracy_RF.consumersAccuracy().getInfo()

[[0.4647887323943662,
  0.46153846153846156,
  0.5,
  0.5087719298245614,
  0.2549019607843137]]

In [14]:
###############################################################################################SVM

In [15]:
classifier_SVM = ee.Classifier.libsvm().train(**params)

In [16]:
image = dxal_s2_image.toFloat()
classified_SVM = image.select(feature_names).classify(classifier_SVM)
vis_class = {
    'min':'0',
    'max':'4',
    'palette': ['006400' ,'ffbb22', 'ffff4c', 'f096ff', 'fa0000']
}
dxal.addLayer(classified_SVM,vis_class,'SVM')
# legend_keys = ['桦树林','落叶松林','针叶混交林','阔叶混交林','针阔混交林']
# colors = ['006400' ,'ffbb22', 'ffff4c', 'f096ff', 'fa0000'] 
# dxal.add_legend(
#     title="varieties",labels=legend_keys,colors=colors, height='100px',position='bottomright'
# )
dxal

Map(bottom=11143.0, center=[51.87, 124.36], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…

In [17]:
validationSample_SVM = yd2.classify(classifier_SVM)
validationAccuracy_SVM = validationSample_SVM.errorMatrix(label, 'classification')
validationAccuracy_SVM.getInfo()

[[22, 9, 4, 17, 16],
 [18, 26, 5, 8, 20],
 [3, 4, 4, 4, 3],
 [18, 2, 2, 19, 9],
 [21, 9, 1, 8, 22]]

In [18]:
validationAccuracy_SVM.kappa().getInfo()

0.1492143004185823

In [19]:
validationAccuracy_SVM.accuracy().getInfo()

0.33941605839416056

In [20]:
validationAccuracy_SVM.producersAccuracy().getInfo()

[[0.3235294117647059],
 [0.33766233766233766],
 [0.2222222222222222],
 [0.38],
 [0.36065573770491804]]

In [21]:
validationAccuracy_SVM.consumersAccuracy().getInfo()

[[0.2682926829268293, 0.52, 0.25, 0.3392857142857143, 0.3142857142857143]]

In [22]:
###########################################################################串行集成

In [23]:
feature_names_maxent = ['elevation', 'slope', 'aspect',
                             'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A','B11', 'B12',
                             'NDVI', 'TVI', 'MTCI', 'NBR', 'IRECI', 'MCARI2', 'REP', 'CIrededge', 'LSWI', 'NDRE', 'Chl_NDI', 'NDSVI', 'NDTI', 'NDSI', 'NDBI', 'NDWI',
                             'NDVI_May', 'NDVI_June', 'NDVI_July', 'NDVI_Aug', 'NDVI_Sep',
                             'B2_con', 'B2_diss', 'B2_var', 'B2_savg', 'B2_shade', 'B2_corr', 
                             'B3_con', 'B3_diss', 'B3_var', 'B3_savg', 'B3_shade', 'B3_corr', 
                             'B4_con', 'B4_diss', 'B4_var', 'B4_savg', 'B4_shade', 'B4_corr', 
                             'B5_con', 'B5_diss', 'B5_var', 'B5_savg', 'B5_shade', 'B5_corr', 
                             'B6_con', 'B6_diss', 'B6_var', 'B6_savg', 'B6_shade', 'B6_corr', 
                             'B7_con', 'B7_diss', 'B7_var', 'B7_savg', 'B7_shade', 'B7_corr', 
                             'B8_con', 'B8_diss', 'B8_var', 'B8_savg', 'B8_shade', 'B8_corr', 
                             'B8A_con', 'B8A_diss', 'B8A_var', 'B8A_savg', 'B8A_shade', 'B8A_corr', 
                             'B11_con', 'B11_diss', 'B11_var', 'B11_savg', 'B11_shade', 'B11_corr', 
                             'B12_con', 'B12_diss', 'B12_var', 'B12_savg', 'B12_shade', 'B12_corr', 
                             'brightness', 'greenness', 'wetness',
                             'birch', 'larch', 'Mixed_2', 'mixed_3', 'mixed_4']
# feature_names_maxent = ['NDVI_July','B11_savg','elevation','NDVI_Sep','B8_savg',
#                     'wetness','NDVI_May','B2_shade','B2_var','aspect',
#                     'NDWI','NDVI_June','B3_shade','B5_savg','B3',
#                     'B6_corr','slope','B7_corr','B8A_corr','NDSI',
#                     'birch', 'larch', 'Mixed_2', 'mixed_3', 'mixed_4'] 
                    # 'TVI','B2_corr','B8A_shade','B11','B12_var',
                    # 'B4_shade','B12','B3_savg','NDRE','MCARI2',
                    # 'B3_con','REP','B6_savg','B8_shade','Chl_NDI',
                    # 'B11_shade','B6','B7_shade','B5_corr','B4']
label = 'varieties'

In [24]:
params_maxent = {
    'features': yd3,
    'classProperty': label,
    'inputProperties': feature_names_maxent
}

In [25]:
classifier_RF_maxent= ee.Classifier.smileRandomForest(150).train(**params_maxent)

In [20]:
image = dxal_s2_image.toFloat()
classified_RF_maxent = image.select(feature_names_maxent).classify(classifier_RF_maxent)
vis_class = {
    'min':'0',
    'max':'4',
    'palette': ['006400' ,'ffbb22', 'ffff4c', 'f096ff', 'fa0000']
}
dxal.addLayer(classified_RF_maxent,vis_class,'RF_maxent')
legend_keys = ['桦树林','落叶松林','针叶混交林','阔叶混交林','针阔混交林']
colors = ['006400' ,'ffbb22', 'ffff4c', 'f096ff', 'fa0000'] 
dxal.add_legend(
    title="varieties",labels=legend_keys,colors=colors, height='230px',position='bottomright'
)
dxal

Map(bottom=11143.0, center=[51.87, 124.36], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…

In [26]:
validationSample_RF_maxent = yd4.classify(classifier_RF_maxent)
validationAccuracy_RF_maxent = validationSample_RF_maxent.errorMatrix(label, 'classification')
validationAccuracy_RF_maxent.getInfo()

[[36, 9, 0, 13, 10],
 [10, 47, 0, 8, 12],
 [2, 12, 2, 0, 2],
 [15, 7, 0, 22, 6],
 [12, 26, 0, 8, 15]]

In [27]:
validationAccuracy_RF_maxent.kappa().getInfo()

0.2676373355841598

In [28]:
validationAccuracy_RF_maxent.accuracy().getInfo()

0.44525547445255476

In [29]:
validationAccuracy_RF_maxent.producersAccuracy().getInfo()

[[0.5294117647058824],
 [0.6103896103896104],
 [0.1111111111111111],
 [0.44],
 [0.2459016393442623]]

In [30]:
validationAccuracy_RF_maxent.consumersAccuracy().getInfo()

[[0.48, 0.46534653465346537, 1, 0.43137254901960786, 0.3333333333333333]]

In [ ]:
############################################################特征筛选

In [ ]:
#######################################################RF随机森林重要性排序
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
trained = ee.Classifier.smileRandomForest(150).train(**{
  'features':yd1,
  'classProperty':'varieties',
  'inputProperties':feature_names
});
importance = trained.explain().get('importance').getInfo()
a = sorted(importance.items(),key=lambda d:d[1],reverse = True)
a2 = [(elem1,elem2) for elem1,elem2 in a]
################################################
plt.figure(figsize=(15,5),dpi=100)
plt.bar(*zip(*a2),alpha=1,width=0.6); 
plt.xlabel('Bands')
plt.ylabel('Importance')
plt.xticks(rotation='vertical')
plt.gca().xaxis.set_tick_params(pad=5)  # 调整标签与坐标轴的距离
plt.show()
######################
# colors = ['red', 'green', 'blue', 'yellow', 'orange']
# cmap = cm.get_cmap('tab10', len(a))
# plt.figure(figsize=(15, 5), dpi=100)
# plt.bar(*zip(*a2), alpha=1, width=0.6, color=cmap(range(len(a))))
# plt.xlabel('Bands')
# plt.ylabel('Importance')
# plt.xticks(rotation='vertical')
# plt.gca().xaxis.set_tick_params(pad=5)
# plt.show()

In [ ]:
# 特征重要性排序
sorted_importance = sorted(importance.items(), key=lambda d: d[1], reverse=True)
# 记录精度和特征数量的列表
accuracy_list = []
feature_count_list = []
# 逐个选择特征并建模
for i in range(len(sorted_importance)):
    selected_features = [item[0] for item in sorted_importance[:i+1]]
    selected_features.append('varieties')
    selected_classifier = ee.Classifier.smileRandomForest(150).train(**{
        'features': yd1.select(selected_features),
        'classProperty': label,
        'inputProperties': selected_features
    })
    # 使用验证集计算精度
    validation_accuracy = yd2.classify(selected_classifier).errorMatrix(label, 'classification').accuracy().getInfo()
    accuracy_list.append(validation_accuracy)
    feature_count_list.append(i+1)  # 添加特征数量到列表中
# 绘制精度与特征数量的折线图
plt.figure(figsize=(8, 6))
plt.plot(feature_count_list, accuracy_list, linestyle='-', marker='None')
plt.xticks(range(0, len(sorted_importance) + 1, 5))
plt.xlabel('Number of Features')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Number of Features')
plt.xlim(0, len(sorted_importance) + 1)  # 设置X轴范围
plt.show()

In [ ]:
#####################################################################maxent验证

In [5]:
lc = ee.Image('projects/gaogao-gee-learn/assets/maxent_fin')
classValues = [0,1,2,3,4]
remapValues = ee.List.sequence(0, 4)
label = 'varieties'
lc = lc.remap(classValues, remapValues).rename(label).toByte()

In [ ]:
sample = img.addBands(lc).stratifiedSample(**{
  'numPoints': 100,
  'classBand': label,
  'region': img.geometry(),
  'scale': 10,
  'geometries': True
})


In [6]:
import numpy as np
import geopandas as gpd
from osgeo import gdal

def sample_classification_results(classification_file, validation_file):
    classification_dataset = gdal.Open(classification_file, gdal.GA_ReadOnly)
    classification_data = classification_dataset.GetRasterBand(1).ReadAsArray()

    validation_data = gpd.read_file(validation_file)
    validation_labels = validation_data["varieties"]

    sampled_classification = []
    sampled_validation = []

    for index, row in validation_data.iterrows():
        x = int(row.geometry.x)
        y = int(row.geometry.y)

        classification_label = classification_data[y, x]
        validation_label = validation_labels[index]

        if classification_label >= 0 and validation_label >= 0:
            sampled_classification.append(classification_label)
            sampled_validation.append(validation_label)

    return np.array(sampled_classification), np.array(sampled_validation)

def calculate_confusion_matrix(sampled_classification, sampled_validation, num_classes):
    confusion_matrix = np.zeros((num_classes, num_classes), dtype=np.int)

    for i in range(len(sampled_classification)):
        predicted = sampled_classification[i]
        actual = sampled_validation[i]
        confusion_matrix[actual, predicted] += 1

    return confusion_matrix

def calculate_overall_accuracy(confusion_matrix):
    diagonal_sum = np.trace(confusion_matrix)
    total_sum = np.sum(confusion_matrix)
    overall_accuracy = diagonal_sum / total_sum

    return overall_accuracy

# 示例用法
classification_file = r"E:\maxent_tif\max\maxent_fin.tif"
validation_file = r"C:\Users\R6meg\Desktop\tree\GEE_test.shp"
num_classes = 5

sampled_classification, sampled_validation = sample_classification_results(classification_file, validation_file)
confusion_matrix = calculate_confusion_matrix(sampled_classification, sampled_validation, num_classes)
overall_accuracy = calculate_overall_accuracy(confusion_matrix)

print("Confusion Matrix:")
print(confusion_matrix)
print("Overall Accuracy:", overall_accuracy)

IndexError: index 5881564 is out of bounds for axis 0 with size 14988

In [7]:
import numpy as np
import geopandas as gpd
from osgeo import gdal

def calculate_confusion_matrix(classification_file, validation_file, num_classes):
    classification_dataset = gdal.Open(classification_file, gdal.GA_ReadOnly)
    classification_data = classification_dataset.GetRasterBand(1).ReadAsArray()

    validation_data = gpd.read_file(validation_file)
    validation_labels = validation_data["varieties"]

    confusion_matrix = np.zeros((num_classes, num_classes), dtype=np.int)

    for index, row in validation_data.iterrows():
        x = int(row.geometry.x)
        y = int(row.geometry.y)

        classification_label = classification_data[y, x]
        validation_label = validation_labels[index]

        if classification_label >= 0 and validation_label >= 0:
            confusion_matrix[validation_label, classification_label] += 1

    return confusion_matrix

# 示例用法
classification_file = r"E:\maxent_tif\max\maxent_fin.tif"
validation_file = r"C:\Users\R6meg\Desktop\tree\GEE_test.shp"
num_classes = 5

confusion_matrix = calculate_confusion_matrix(classification_file, validation_file, num_classes)

print("Confusion Matrix:")
print(confusion_matrix)

IndexError: index 5881564 is out of bounds for axis 0 with size 14988